In [1]:
import pandas as pd

# docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
# print(len(docs))
papers = pd.read_csv('./data_face/data_xuli_16112023.csv')
print(papers.columns)
# papers.rename(columns={'body': 'paper_text'}, inplace=True)
# convert type of column
papers['text'].astype(str)
# Print head
print(papers.head())
print(papers.shape)

Index(['post_id', 'text', 'time', 'timestamp', 'comments', 'shares',
       'post_url', 'user_id', 'username', 'reaction_count', 'thích',
       'yêu thích', 'haha', 'wow', 'buồn', 'phẫn nộ', 'thương thương',
       'comments_full', 'commenter_name'],
      dtype='object')
           post_id                                               text  \
0  714667314027730  #cfs17338\nChuyện về ktx ạ :((\nMình là người ...   
1  714236537404141  #cfs17337\nChào các bạn, đêm khuya không ngủ đ...   
2  713014190859709  #cfs17336\nMình là sinh viên năm cuối. Năm cuố...   
3  713013604193101  #cfs17335\nChuyện thang máy đi từ tầng G đã kh...   
4  713013097526485  #cfs17334\nChuyện mình muốn nói hôm nay là vấn...   

                  time   timestamp  comments  shares  \
0  2023-11-16 11:01:33  1700107293        23       0   
1  2023-11-15 20:02:08  1700053328       120       1   
2  2023-11-15 15:02:03  1700035323        54       0   
3  2023-11-15 10:13:03  1700017983       312       3   
4  2023

In [2]:
import numpy as np
# duplicated
print(papers.duplicated().sum())
papers.drop_duplicates(inplace=True)

# drop nan
papers.dropna(axis=0, inplace=True, how="any")
# drop inf
papers.replace([np.inf, -np.inf], np.nan, inplace=True)
papers.replace('', np.nan, inplace=True)
papers.dropna(inplace=True,how='any',axis=0)
# Remove the columns
try:
    papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)
except:
    pass

# Print out the first rows of papers
print(papers.head())
print(papers.info())
papers.reset_index(inplace=True, drop=True)

0
           post_id                                               text  \
0  714667314027730  #cfs17338\nChuyện về ktx ạ :((\nMình là người ...   
1  714236537404141  #cfs17337\nChào các bạn, đêm khuya không ngủ đ...   
2  713014190859709  #cfs17336\nMình là sinh viên năm cuối. Năm cuố...   
3  713013604193101  #cfs17335\nChuyện thang máy đi từ tầng G đã kh...   
4  713013097526485  #cfs17334\nChuyện mình muốn nói hôm nay là vấn...   

                  time   timestamp  comments  shares  \
0  2023-11-16 11:01:33  1700107293        23       0   
1  2023-11-15 20:02:08  1700053328       120       1   
2  2023-11-15 15:02:03  1700035323        54       0   
3  2023-11-15 10:13:03  1700017983       312       3   
4  2023-11-14 14:10:00  1699945800        16       0   

                                            post_url          user_id  \
0  https://facebook.com/PtitCfsOfficial/posts/714...  100064534036414   
1  https://facebook.com/PtitCfsOfficial/posts/714...  100064534036414   
2  

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from underthesea import word_tokenize, sent_tokenize
from underthesea import ner
import torch

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

def ner_results(text) -> dict[str, list[str]]: # dict[str, list[str]]: key là tên thực thể, value là list các từ thuộc thực thể đó
    sentences = sent_tokenize(text)
    # print(sentences)
    sentences = [s for s in sentences if len(s) > 0]

    list_ner = []
    results_sentences = []

    for s in sentences:
        if len(s) > 0:
            tensor_input = torch.tensor([tokenizer.encode(s)])
            # print(tensor_input.shape)
            if(tensor_input.shape[1] < 512):
                # print(preprocess(s))
                # print(nlp(s))
                if len(nlp(s)) > 0:
                    word = ""
                    original_word = ""
                    # s = preprocess(s)
                    for i in nlp(s):
                        if 'B-' in i['entity']:
                            word = i['word']
                            original_word = i['word']
                        elif 'I-' in i['entity']:
                            original_word += " " + i['word']
                            if i['word'] != "-":
                                word += " " + i['word']
                        elif 'E-' in i['entity']:
                            word += " " + i['word']
                            original_word += " " + i['word']
                            list_ner.append({'entity': i['entity'].split('-')[1], 'word': word})
                            word = ""
                            original_word = ""
                        elif 'S-' in i['entity']:
                            list_ner.append({'entity': i['entity'].split('-')[1], 'word': i['word']})
                            word = ""
                            original_word = ""
                    if word != "":
                        list_ner.append({'entity': i['entity'].split('-')[1], 'word': word})
                        # print(word)
                        s = s.replace(original_word, "_".join(word.split(" ")).lower())
                        # print(s)
                        results_sentences.append(s)
                        word = ""
                        original_word = ""
                #
                list_ner += nlp(s)
                # print(nlp(preprocess(s)))
                # print("=========================================")
            else:
                print("sentence too long")

    # print(list_ner)
    dict_ner = {}
    for i in list_ner:
        if i['entity'] in dict_ner:
            dict_ner[i['entity']].append(i['word'])
        else:
            dict_ner[i['entity']] = [i['word']]

    results = {}
    for key, value in dict_ner.items():
        if '-' not in key:
            # print(key + ": " + str(value))
            results[key] = value
    
    print(results_sentences)
    return results, " ".join(results_sentences)

ner_results_location_ = []
ner_results_person_ = []
ner_results_organization_ = []
doc = []
for i in range(len(papers['text'])):
    results, doc_ = ner_results(papers['text'][i])
    dict_ner = results
    doc.append(doc_)
    print(i)
    if 'LOCATION' not in dict_ner:
        dict_ner['LOCATION'] = []
    if 'PERSON' not in dict_ner:
        dict_ner['PERSON'] = []
    if 'ORGANIZATION' not in dict_ner:
        dict_ner['ORGANIZATION'] = []    
    ner_results_location_.append(dict_ner['LOCATION'])
    ner_results_person_.append(dict_ner['PERSON'])
    ner_results_organization_.append(dict_ner['ORGANIZATION'])
    # print(dict_ner)
    print(doc_)
    
papers['location'] = ner_results_location_
papers['person'] = ner_results_person_
papers['organization'] = ner_results_organization_
papers['doc_ner'] = doc
papers.to_csv('./data_face/data_xuli_16112023_ner.csv', index=False)


2023-11-17 10:45:38.731681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]
0

[]
1

['Khi làm teamwork mình cảm thấy không được tôn trọng lắm thì phải, nhiều lần mình làm xong không được đồng quan điểm với các bạn ấy là các bạn ý bác bỏ một cách phũ phàng ạ ( thậm chí xoá luôn phần mình làm và đưa ý của các bạn ý vào) .Xem thêm Thực sự nhiều khi chỉ muốn làm một mình , chứ làm nhóm đã đau đầu mệt mỏi lại thêm stress vì gặp được những người bạn thiếu tôn trọng vậy 😞\n\n-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\\-\n💥 GỬI CFS TẠI: http://bit.ly/ptitcfshn\nptit Confessions Offical (since 2013)\nEmail: ptitcfs.hanoi@gmail.com\n\n#ptitconfessions #ptitcfs #ptitcogivui #sinhvienptit #congdongptit #ptitmaidinh']
2
Khi làm teamwork mình cảm thấy không được tôn trọng lắm thì phải, nhiều lần mình làm xong không được đồng quan điểm với các bạn ấy là các bạn ý bác bỏ một cách phũ phàng ạ ( thậm chí xoá luôn phần mình làm và đưa ý của các bạn ý vào) .Xem thêm Thực sự nhiều khi chỉ muốn làm một mình , chứ làm nhóm đã đ

In [2]:
papers.head()

,post_id,text,time,timestamp,comments,shares,post_url,user_id,username,reaction_count,thích,yêu thích,haha,wow,buồn,phẫn nộ,thương thương,comments_full,commenter_name
0,714667314027730,#cfs17338\nChuyện về ktx ạ :((\nMình là người ...,2023-11-16 11:01:33,1700107293,23,0,https://facebook.com/PtitCfsOfficial/posts/714...,100064534036414,PTIT Confessions,79,71.0,1.0,3.0,2.0,2.0,0.0,0.0,"Phí Thúy 1h đêm đừng có học nữa Bạn aim bn, mì...","Nguyễn Thị Thanh Thảo, Nguyễn Tuấn Anh, Cường ..."
1,714236537404141,"#cfs17337\nChào các bạn, đêm khuya không ngủ đ...",2023-11-15 20:02:08,1700053328,120,1,https://facebook.com/PtitCfsOfficial/posts/714...,100064534036414,PTIT Confessions,178,78.0,0.0,74.0,1.0,24.0,1.0,0.0,Minhh Hạnh ô b à Ý là bạn với ny giận nhau là ...,"Vũ Hiếu, Sky Otaku Sama, Ngọc Quỳnh, Trang, Ng..."
2,713014190859709,#cfs17336\nMình là sinh viên năm cuối. Năm cuố...,2023-11-15 15:02:03,1700035323,54,0,https://facebook.com/PtitCfsOfficial/posts/713...,100064534036414,PTIT Confessions,160,123.0,3.0,4.0,1.0,29.0,0.0,0.0,qua team tôi cho bạn mặc sức làm từ đầu tới cu...,"Phan Huy, Nguyễn Khôi Nguyên, Hiếu Hóm Hỉnh, Đ..."
3,713013604193101,#cfs17335\nChuyện thang máy đi từ tầng G đã kh...,2023-11-15 10:13:03,1700017983,312,3,https://facebook.com/PtitCfsOfficial/posts/713...,100064534036414,PTIT Confessions,358,136.0,1.0,46.0,4.0,169.0,1.0,1.0,Hên sáng nay k đi học =))) Thuy Luong Thi ôi x...,"Hoàn Yên, Huệ Thanh Trần, Như Thắng, Trường Gi..."
4,713013097526485,#cfs17334\nChuyện mình muốn nói hôm nay là vấn...,2023-11-14 14:10:00,1699945800,16,0,https://facebook.com/PtitCfsOfficial/posts/713...,100064534036414,PTIT Confessions,100,80.0,10.0,0.0,0.0,10.0,0.0,0.0,Đức Nam hình như chửi a với m Dương Phạm đừng ...,"Lý Trọng Nghĩa, Hoàng Đức Khiêm, Trang Trần, H..."


In [3]:
positive = []
negative = []
p = ['like', 'love', 'haha', 'wow']
n = ['sad', 'angry']
for i in range(len(papers)):
    like = papers['thích'][i]
    love = papers['yêu thích'][i]
    haha = papers['haha'][i]
    wow = papers['wow'][i]
    sad = papers['buồn'][i]
    angry = papers['phẫn nộ'][i]
    l = {'like': like, 'love': love, 'haha': haha, 'wow': wow, 'sad': sad, 'angry': angry}
    l = dict(sorted(l.items(), key=lambda item: item[1], reverse=True))
    key = list(l.keys())
    print(key)
    if key[0] in n:
        negative.append(1)
        positive.append(0)
    elif key[1] in n:
        negative.append(1)
        positive.append(0)
    elif key[0] in p:
        positive.append(1)
        negative.append(0)

papers['negative'] = negative
papers['positive'] = positive
papers.to_csv('./data_face/data_xuli_16112023_ner_sentiment.csv', index=False)
    

['like', 'haha', 'wow', 'sad', 'love', 'angry']
['like', 'haha', 'sad', 'wow', 'angry', 'love']
['like', 'sad', 'haha', 'love', 'wow', 'angry']
['sad', 'like', 'haha', 'wow', 'love', 'angry']
['like', 'love', 'sad', 'haha', 'wow', 'angry']
['like', 'haha', 'love', 'sad', 'wow', 'angry']
['haha', 'like', 'sad', 'love', 'wow', 'angry']
['like', 'haha', 'sad', 'love', 'wow', 'angry']
['like', 'haha', 'sad', 'love', 'wow', 'angry']
['like', 'haha', 'wow', 'love', 'sad', 'angry']
['like', 'love', 'haha', 'wow', 'sad', 'angry']
['like', 'sad', 'haha', 'love', 'wow', 'angry']
['like', 'love', 'sad', 'haha', 'wow', 'angry']
['like', 'haha', 'sad', 'wow', 'angry', 'love']
['like', 'haha', 'sad', 'love', 'wow', 'angry']
['like', 'haha', 'wow', 'love', 'sad', 'angry']
['like', 'love', 'haha', 'sad', 'wow', 'angry']
['haha', 'like', 'sad', 'love', 'wow', 'angry']
['like', 'haha', 'sad', 'love', 'wow', 'angry']
['like', 'haha', 'sad', 'love', 'wow', 'angry']
